# MOM adaptation of Dantzig Selector

Source of the algorithm applied : https://apps.dtic.mil/dtic/tr/fulltext/u2/a619308.pdf

Let X the matrix of our observation and Y the target. $X_j$ denotes the j-th column (variable) of X and $x_i$ the i-th row of X.
$$\forall j \in (1,n) \quad x_j \sim P$$

The Dantzig Selector is the solution to the following problem of optimisation : 
$$  \widehat{\beta} \in arg \min_{\beta }{\|\beta\|_1} \quad  s.c : \|D^{-1}X^T(X\beta - Y) \|_\infty \le \delta  $$
With $\delta$ a fixed parameter of the model and D a diagonal matrix with $D_{i,i} = \|X_i\|_2$



In [43]:
import numpy as np

In [44]:
def norm_1(x):
    
    return np.sum(np.abs(x))

def prox(u , lamb):
    
    n = len(u)
    prox = np.zeros((n,1))
    
    for i in range(n) :
        
        prox[i,0] = np.sign(u[i,0]) * np.max(np.abs(u[i,0]) - lamb , 0)
    
    return prox

In [45]:
def dantzig_selector(Y , X , delta , alpha , tol):
    
    n , p = np.shape(X)
    D_inv = np.diag([1/np.linalg.norm(X[:,i]) for i in range(p)])
    A = D_inv @ X.T @ X
    b = D_inv @ X.T @ Y
    
    lamb = (0.999 * alpha) / np.linalg.norm(A)**2
    
    t , t_prev = np.zeros((p , 1)) , np.zeros((p , 1))
    to = np.zeros((p , 1))
    
    #Stage 1 : 
    for i in range(iter_max):
        
        to = prox(A @ (2 * t - t_prev) + to - b , delta)
        t_prev = t
        t = prox( t - (lamb / alpha) * A.T @ to , 1 / alpha)
        
    #Stage 2 : 
    
    val_sel = np.where(np.abs(t) < tol)[0]
    v = np.linalg.solve(X[:,var_sel].T @ X[:,var_sel] , X[:,var_sel].T @ Y )
    
    sol = np.zeros((p,1))
    sol[val_sel] = v
    
    return sol